In [62]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go


In [63]:
df1 = pd.read_csv('DataSet3_1.csv', sep = ',')
df2 = pd.read_csv('DataSet3_2.csv', sep = ';')

df1[["Timestamp"]] = df1[["Timestamp"]].apply(pd.to_datetime)
df1.head()

,Timestamp,OAT (F),Power (kW)
0,2010-01-01 01:15:00,51,165.1
1,2010-01-01 01:30:00,51,151.6
2,2010-01-01 01:45:00,51,146.9
3,2010-01-01 02:00:00,51,153.7
4,2010-01-01 02:15:00,51,153.8


In [64]:
df2.head()

,Data,Dollar,Oil Brent
0,01.01.2020,"61,9057","66,39"
1,09.01.2020,"61,9057","65,34"
2,10.01.2020,"61,234","65,02"
3,11.01.2020,"61,2632","64,84"
4,14.01.2020,"60,9474","64,42"


In [65]:
df1['Timestamp'] =   (df1['Timestamp']).astype(np.int64)/1000000000 #нс -> c
df1.head()

,Timestamp,OAT (F),Power (kW)
0,1.262308e+09,51,165.1
1,1.262309e+09,51,151.6
2,1.262310e+09,51,146.9
3,1.262311e+09,51,153.7
4,1.262312e+09,51,153.8


## Линейная интерполяция

In [66]:

x = df1['Timestamp'][9:25].to_numpy()
y = df1['Power (kW)'][9:25].to_numpy()

x1 = np.arange(x[0],x[15], 1)
y1 = np.interp(x1, x, y)

fig = go.Figure()

fig.add_trace(go.Scatter(
                    x=x,
                    y=y,
                    name='Исходные данные',
                    mode='markers')
)

fig.add_trace(go.Scatter(
                    x=x1,
                    y=y1,
                    name='Линейная интерполяция',
                    )
)

fig.update_layout(title='Потребление энергии',
                    xaxis_title='Время',
                    yaxis_title='кВт')
fig.show()

### Кубический сплайн


In [82]:
from scipy.interpolate import CubicSpline
inter_f =  CubicSpline(x, y)

fig = go.Figure()

fig.add_trace(go.Scatter(
                    x=x,
                    y=y,
                    name='Исходные данные',
                    mode='markers')
)

fig.add_trace(go.Scatter(
                    x=x1,
                    y=inter_f(x1),
                    name='Интерполяция',
                    )
)

fig.update_layout(title='Потребление энергии',
                    xaxis_title='Время',
                    yaxis_title='кВт')
fig.show()


### Интерполяция Лагранжа 

In [83]:
import numpy as np
from scipy.interpolate import lagrange
x = df1['Timestamp'][9:25].to_numpy()
y = df1['Power (kW)'][9:25].to_numpy()

x = x - np.mean(x)
x = x/np.max(x)

y = y - np.mean(y)
y = y/np.max(y)

x1 = np.arange(x[0],x[15]+0.005, 0.005)

poly = lagrange(x, y)
y1 = Polynomial(poly.coef[::-1])(x1)
from numpy.polynomial.polynomial import Polynomial

fig = go.Figure()

fig.add_trace(go.Scatter(
                    x=x,
                    y=y,
                    name='Исходные данные',
                    mode='markers')
)

fig.add_trace(go.Scatter(
                    x=x1,
                    y= y1,
                    name='Интерполяция',
                    )
)

fig.update_layout(title='Потребление энергии',
                    xaxis_title='Время',
                    yaxis_title='кВт')
fig.show()

### Интерполяция Ньютона

In [81]:
x = df1['Timestamp'][9:25].to_numpy()
y = df1['Power (kW)'][9:25].to_numpy()

x1 = np.arange(x[0],x[15], 1)

def divided_diff(x, y):

    n = len(y)
    coef = np.zeros([n, n])
    # the first column is y
    coef[:,0] = y
    
    for j in range(1,n):
        for i in range(n-j):
            coef[i][j] = (coef[i+1][j-1] - coef[i][j-1]) / (x[i+j]-x[i])
            
    return coef

def newton_poly(coef, x_data, x):

    n = len(x_data) - 1 
    p = coef[n]
    for k in range(1,n+1):
        p = coef[n-k] + (x -x_data[n-k])*p
    return p

a_s = divided_diff(x, y)[0, :]
y1= newton_poly(a_s, x, x1)

fig = go.Figure()

fig.add_trace(go.Scatter(
                    x=x,
                    y=y,
                    name='Исходные данные',
                    mode='markers')
)

fig.add_trace(go.Scatter(
                    x=x1,
                    y= y1,
                    name='Интерполяция',
                    )
)

fig.update_layout(title='Потребление энергии',
                    xaxis_title='Время',
                    yaxis_title='кВт')
fig.show()

### Задание
Взять не менее 10 значений и построить по ним интерполяционные кривые по формуле Лагранжа, по формуле Ньютона, с использованием сплайнов (интерполяция кубическим сплайном).

### Данные
* DataSet3_1 – данные о частоте и потребляемой мощности электрической энергии (раноотстоящие узлы).
* DataSet3_2 – данные о стоимости доллара и нефти (не равноотстоящие узлы)